In [3]:
import sys
sys.path.insert(0, '../')

import numpy as np
import os
import timeit


In [14]:
from queries import Queries
from queries import concatenate
from queries import find_constant_features
from click_simulator import CascadeModel

In [10]:
# Load Queries
queries_list = []
for name in ['train.txt', 'test.txt', 'vali.txt']:
    data_dir = '../../datasets/MSLR10K/Fold1/' + name
    queries = Queries.load_from_text(data_dir)
    cls = find_constant_features(queries)
    queries.adjust(remove_features=cls, purge=True, scale=True)
    queries.save(data_dir[:-3]+'dat')
    queries_list.append(queries)
query_all = concatenate(queries_list)
query_all.save('../../datasets/MSLR10K.dat')

In [12]:
data_dir = '../../datasets/MSLR10K.dat'
queries = Queries.load_from_text(data_dir)

'../../datasets/MSLR10K/Fold1/test.'

In [17]:
simulator = CascadeModel('perfect')

In [20]:
start = timeit.default_timer()
n_clicks = 0
n_docs = 0
iteration = int(1e4)
for i in range(iteration):
    query = queries[np.random.choice(len(queries))]
    while query.document_count < 10 or query.relevance_scores.max() == 0:
        print(1)
        query = queries[np.random.choice(len(queries))]
    y =  query.relevance_scores[np.random.permutation(len(query))]
    click = simulator.get_click(y)
    n_clicks += sum(click)
    n_docs += len(click)

print("running time of %d sessions: %f" %(iteration, timeit.default_timer()-start))
print("number of clicks: %d" % n_clicks)
print("number of docs: %d" % n_docs)

running time of 10000 sessions: 3.089127
number of clicks: 7130
number of docs: 10000
